In [1]:
#%reset -f
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import sympy as sp
import helper_functions as hf

## Introduction

This notebook contains the sympy code to derive the explicit analytical perturbative expressions discussed in [add reference once the paper is on arXiv], which in the following we refer to as Ref. [1].

In this notebook, we symbolically calculate
* the coefficients of the normalization-preserving propagator (NPP; Eq. (23) in Ref. [1])
* the coefficients of the positivity-preserving propagator (PPP; Eq. (41) in Ref. [1]), and
* the first few perturbative moments based on the NPP (Eq. (51) in Ref. [1]).

We save the symbolic results to text files, which are stored in the current folder. 

The files we generate here are in the exact form that the module PySTFP uses. This means that after modifying the code below (e.g. to calculate higher order terms), and saving the results, one can use the text files to modify e.g. the maximal order perturbation theory available in PySTFP. For this, the text files generated here need to be copied the folder **PySTFP/sympy_definitions** (and then the module needs to be re-installed).

## Normalization-preserving propagator (NPP)

We want to calculate the coefficients $\tilde{\mathcal{Q}}_k$ from Eq. (23) of Ref. [1].

For this, we recursively solve Eq. (30) of Ref. [1] with increasing $k=0, 1, 2, ..$.

In [2]:
As = sp.symbols('A_0:%d'%15,real=True)
Ds = sp.symbols('D_0:%d'%15,real=True)

x, y = sp.symbols('x y',real=True)
n = sp.Symbol('n', integer=True ,nonzero=True)
R = sp.Symbol(r'\tilde{R}', positive=True,nonzero=True)
epsilon = sp.Symbol('\epsilon', positive=True,nonzero=True)

P0 = 1/sp.sqrt(2*sp.pi) * sp.exp(-x**2/2) # this is Eq. (24) of Ref. [1]

In [3]:
# For k = 0, we have Q_0 = 1:
list_of_Q = [1]

We first implement the solution for a monomial inhomogeneity

\begin{equation}
\partial_{\tilde{x}}^2 f
- \tilde{x} \partial_{\tilde{x}} f
- k f
= \tilde{x}^l,
\end{equation}

which is given in Eq. (A2) of Ref. [1]

In [4]:
def get_inhomogenous_solution_for_monomial(k, # order of polynomial (prefactor of last term on right-hand side)
                                           l, # power on right-hand side
                                           verbose=False):
    #
    Q_coeffs = sp.symbols('Q_0:%d'%(l+1))
    #
    Q_poly = 0
    for i in range(l+1):
        Q_poly += Q_coeffs[i]*x**i
    #
    LHS = sp.simplify( sp.diff(Q_poly,x,2)  \
                        - x*sp.diff(Q_poly,x,1) \
                        - k*Q_poly )
    RHS = x**l
    solution =  sp.solve(LHS-RHS, Q_coeffs)
    #
    if len(solution) == 1:
        if verbose:
            print("Solution is constant polynomial")
        return solution[0]
    #
    for i in range(l+1):
        try:
            Q_poly = Q_poly.subs(Q_coeffs[i],solution[Q_coeffs[i]])
        except KeyError:
            print("Warning: Key {0} is not specified. Using 0.".format(Q_coeffs[i]))
            Q_poly = Q_poly.subs(Q_coeffs[i],0)
    return Q_poly

test = get_inhomogenous_solution_for_monomial(k=4,l=4)
test

-x**4/8 - x**2/4 - 1/8

We now implement functions to solve Eq. (30) at order $k$, assuming we have solved it to order $k-1$.

In [5]:
# Now we implement the calculation of the right-hand side of Eq. (30), 
# for known list_of_Q = [Q_0, Q_1, .., Q_{k-1}] with k elements
def get_polynomial_on_RHS( # RHS = right-hand side
                        list_of_Q,
                        verbose=False):
    #
    k = len(list_of_Q) # at the k-th iteraction, we have len(list_of_Q) = k
    #
    RHS = 0
    #
    # First term
    for l in range(0,k):
        RHS += As[l] * ( 
                    sp.diff(x**l * list_of_Q[k-1-l], x, 1) 
                    - x**(l+1) * list_of_Q[k-1-l]
                         )
    # Second term
    for l in range(1,k+1):
        RHS -= Ds[l] * (
                    sp.diff(x**l * list_of_Q[k-l], x, 2) 
                    - 2 * x * sp.diff(x**l * list_of_Q[k-l], x, 1) 
                    + x**l * ( x**2 - 1 ) * list_of_Q[k-l]
                     )
    #
    RHS = sp.expand(RHS)
    #
    return RHS

# get polynomial coefficients from a sympy expression
def get_polynomial_coefficients(input_polynomial):
    poly_coeffs = sp.Poly(input_polynomial,x).all_coeffs()
    poly_coeffs = poly_coeffs[::-1]
    return poly_coeffs # [0] = lowest order

# Solve Eq. (30) at order k for a polynomial
# RHS_coeffs[0] * x**0 + RHS_coeffs[1] * x**1 + .. 
# on the right-hand side
def get_current_Q(k, # 
                  RHS_coeffs # coefficients of polynomial on RHS
                 ):
    #
    current_Q = 0
    #
    for i,cur_coeff in enumerate(RHS_coeffs):
        if cur_coeff != 0: # if cur_coeff, then we don't need to add anything
            # to current_Q
            cur_poly = get_inhomogenous_solution_for_monomial(k=k,l=i)
            current_Q += cur_coeff * cur_poly
    #
    return sp.expand(current_Q)

In [6]:
def get_next_order(check_solution=True,
                evaluate_integral=True):
    global list_of_Q # we will append to this list
    #
    # the order we get is equal to the current length of list_of_Q:
    k = len(list_of_Q) #
    print('k =',k)
    # for example, if len(list_of_Q) == 1, then list_of_Q = [1]
    # and only contains the zeroth-order result.
    #
    # get polynomial on the right-hand side of the equation
    polynomial_on_RHS = get_polynomial_on_RHS(list_of_Q=list_of_Q)
    #
    polynomial_coefficients = get_polynomial_coefficients(polynomial_on_RHS)
    #
    Q = get_current_Q(k=k,
                   RHS_coeffs=polynomial_coefficients)
    #
    if check_solution:
        result_of_check = sp.simplify( sp.diff(Q,x,2) \
             - x*sp.diff(Q,x,1) \
             - k*Q  \
             - polynomial_on_RHS)
        if result_of_check == 0:
            print("Solution validated")
        else:
            raise RuntimeError("Solution invalid")
    #
    if evaluate_integral:
        integral = 0
        D0_ = sp.symbols('D_0',real=True,positive=True) # dummy
        for i,e in enumerate(Q.args):
            current_expression = e*P0
            #
            current_expression = current_expression.subs(Ds[0],D0_)
            current_integral = sp.simplify(sp.integrate(current_expression,
                                    (x,-sp.oo,sp.oo)).doit())
            current_integral = current_integral.subs(D0_,Ds[0])
            #
            integral += current_integral
            integral = sp.expand(integral)
        print("integral = {0}".format(integral))
    #
    list_of_Q.append(Q)
    #

list_of_Q = [1]
for i in range(8):
    get_next_order()

k = 1
Solution validated
integral = 0
k = 2
Solution validated
integral = 0
k = 3
Solution validated
integral = 0
k = 4
Solution validated
integral = 0
k = 5
Solution validated
integral = 0
k = 6
Solution validated
integral = 0
k = 7
Solution validated
integral = 0
k = 8
Solution validated
integral = 0


For the $\tilde{\mathcal{Q}}_k$ that we calculated, we find out 
what the largest appearing index $l$ for $\tilde{\mathcal{A}}_l$, $\tilde{\mathcal{D}}_l$ is:

In [7]:
N_max_D = hf.get_maximal_D(list_of_Q[-1])
N_max_A = hf.get_maximal_A(list_of_Q[-1])
print(N_max_D,N_max_A)

N_max = max( N_max_D,N_max_A ) + 1
N_max

8 7


9

We save the normalization-preserving propagator to a text file

In [8]:
with open('normalization_preserving_propagator.py','w') as f:
    f.write('''
import sympy

# highest appearing values: Dk[{N_max_D}], Ak[{N_max_A}]

N_max = {N_max}

Dk = sympy.symbols('D_0:%d'%N_max)
Ak = sympy.symbols('A_0:%d'%N_max)
xDL = sympy.symbols('\\tilde{{x}}',real=True)
epsilon = sympy.symbols('\epsilon',real=True,nonnegative=True)

P0 = {P0}

Q_dict = {{}}

'''.format(
    N_max_D=N_max_D,
    N_max_A=N_max_A,
    N_max = N_max,
    P0=hf.sympy_expression_to_string(P0))
    )
    #
    for i,e in enumerate(list_of_Q):
        f.write('Q_dict[{0}] = '.format(i))
        f.write(hf.sympy_expression_to_string(e))
        f.write('\n\n')        
    #
    f.write('''
    
P0_lambda = sympy.lambdify( (Dk[0],xDL) , P0)

Q_lambda_dict = {}
for i,item in Q_dict.items():
	Q_lambda_dict[i] = sympy.lambdify( (Dk,Ak,xDL), item)

Q_epsilon_sum = 0
for i,term in Q_dict.items():
	Q_epsilon_sum = Q_epsilon_sum + epsilon**i * term
Q_epsilon_sum_lambda = sympy.lambdify( (Dk,Ak,xDL,epsilon), Q_epsilon_sum)

    ''')
    #

## Positivity-preserving propagator

To calculate the positivity-preserving propagator, Eq. (41) in Ref. [1], 
we first define the power series

\begin{align}
1 + \epsilon \tilde{\mathcal{Q}}_1 
 + \epsilon^2 \tilde{\mathcal{Q}}_2 
 + ...
\end{align}

with new sympy symbols $\tilde{\mathcal{Q}}_k$. We then calculate the
 series Eq. (41), and only in the end substitute 
for the $\tilde{\mathcal{Q}}_k$ the explicit expressions that we have 
calculated in the context of the NPP above.

In [9]:
Q_symbols = sp.symbols('Q_0:%d'%len(list_of_Q))

perturbation = 1
for i,current_Q in enumerate(Q_symbols):
    if i == 0:
        continue
    perturbation += epsilon**i * current_Q

perturbation

Q_1*\epsilon + Q_2*\epsilon**2 + Q_3*\epsilon**3 + Q_4*\epsilon**4 + Q_5*\epsilon**5 + Q_6*\epsilon**6 + Q_7*\epsilon**7 + Q_8*\epsilon**8 + 1

In [10]:
# perturbation = exp(log(perturbation))

exp_perturbation = sp.log(perturbation)
exp_perturbation = sp.series(exp_perturbation,
                        epsilon,0,len(list_of_Q)).removeO()
exp_perturbation = sp.expand(exp_perturbation)

In [11]:
exp_perturbation_coefficients = sp.Poly(exp_perturbation,epsilon).all_coeffs()[::-1]

In [12]:
exp_perturbation_coefficients_substituted = []

for term in exp_perturbation_coefficients:
    #
    for i,Q in enumerate(list_of_Q):
        #
        term = term.subs(Q_symbols[i],Q)
    #
    exp_perturbation_coefficients_substituted.append(sp.expand(term))

Save the result

In [13]:
with open('positivity_preserving_propagator.py','w') as f:
    f.write('''
import sympy

# highest appearing values: Dk[{N_max_D}], Ak[{N_max_A}]

N_max = {N_max}

Dk = sympy.symbols('D_0:%d'%N_max)
Ak = sympy.symbols('A_0:%d'%N_max)
xDL = sympy.symbols('\\tilde{{x}}',real=True)
epsilon = sympy.symbols('\epsilon',real=True,nonnegative=True)

Q_dict = {{}}

'''.format(
        N_max_D = N_max_D,
        N_max_A = N_max_A,
        N_max = N_max)
    )
    #
    for i,expr in enumerate(exp_perturbation_coefficients_substituted):
        f.write('Q_dict[{0}] = '.format(i))
        f.write(hf.sympy_expression_to_string(expr))
        f.write('\n\n')        
    #

## Moments

In dimensionless units, the moments are defined by

\begin{align}
\langle \tilde{x}^n \rangle = \sum_{k=0}^{\infty} \tilde{\epsilon}^k \langle \tilde{x}^n \rangle^{(k)},
\end{align}

with 

\begin{align}
\langle \tilde{x}^n \rangle^{(k)}
&=
\int_{-\infty}^{\infty}d\tilde{x}\,\tilde{x}^n Q_k(\tilde{x}) \tilde{P}^{(0)}(\tilde{x}).
\end{align}

We now calculate the $\langle \tilde{x}^n\rangle^{(k)}$ using the NPP results from above

In [14]:
moment_coefficients = {}

max_moment = 4
max_order= 8

for n in range(max_moment+1):
    #
    print('Calculating moments for <x^{0}>'.format(n))
    #
    moment_coefficients[n] = []
    #
    for k in range(max_order+1):
        #
        # to enforce the assumption that Ds[0] is real and positive,
        # we temporarily replace it by a dummy variable that is created
        # with the assumption 'positive=True'
        dummy_D0 = sp.symbols('D_0',real=True,positive=True)
        integrand = sp.expand(list_of_Q[k]*x**n*P0)
        integrand = integrand.subs(Ds[0],dummy_D0)
        current_integral = sp.integrate(
                                integrand,
                                (x,-sp.oo,sp.oo)
                                        )
        # after the integral, we re-substitute Ds[0] for the dummy variable
        current_integral = current_integral.subs(dummy_D0,Ds[0])
        #
        moment_coefficients[n].append(current_integral)

Calculating moments for <x^0>
Calculating moments for <x^1>
Calculating moments for <x^2>
Calculating moments for <x^3>
Calculating moments for <x^4>


In [15]:
# moment_coefficients[i][j] = <\tilde{x}^i>^{(j)}

moment_coefficients[1][1] # first kramers moyal coefficient

A_0/2

In [16]:
moment_coefficients[2][2] 

A_0**2/4 + A_0*D_1/4 + A_1/2 + D_2/2

In [17]:
with open('moments.py','w') as f:
    f.write('''
import sympy

N_max = {N_max}

Dk = sympy.symbols('D_0:%d'%N_max)
Ak = sympy.symbols('A_0:%d'%N_max)

moments_dict = {{}}

'''.format(N_max = N_max)
    )
    #
    for i,moment_list in moment_coefficients.items():
        #
        f.write('moments_dict[{0}] = {{}}\n\n'.format(i))
        #
        for k, expr in enumerate(moment_list):
            # expr = <x^i>^{(k)}
            f.write('moments_dict[{0}][{1}] = '.format(i,k))
            f.write(hf.sympy_expression_to_string(expr))
            f.write('\n\n')
        f.write('\n\n\n')        
    #